In [2]:
#import libraries needed

import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException as WDE
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt

In [3]:
#select the chrome webdriver, ignore certificate errors, use an incognito tab 
#and the headless argument (accesses driver without actually opening a browser window)

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
#options.add_argument('--incognito')
#options.add_argument('--headless')
driver = webdriver.Chrome("/Users/jordan/Downloads/chromedriver 3", options=options)

In [ ]:
#set url in for loop using an F-string
#get url
#make sure it has the element we need

delay = 10 

rows_data = []

#specify the year
year_number = 2021

for month_number in range(1,13):
    
    if month_number == 1 or 3 or 5 or 7 or 8 or 10 or 12:
        max_days = 31
    
    if month_number == 4 or 6 or 9 or 11:
        max_days = 30
    
    if month_number == 2:
        max_days = 28
        
    for day_number in range(1,max_days+1):

        url = F'https://www.wunderground.com/history/daily/au/brisbane/YBBN/date/{year_number}-{month_number}-{day_number}'
        driver.get(url)
        time.sleep(10)

        # try:
        # WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'observation-table ng-star-inserted')))
            #except TimeoutException:
            #print('error: Loading exceeds delay time')
           #break
        #else:
        soup = BeautifulSoup(driver.page_source, "lxml")
        main_table = soup.find('div', {'class': 'observation-table ng-star-inserted'})
        rows = main_table.find_all('tr', {'class':'mat-row cdk-row ng-star-inserted'})

        for row in rows:
            time_place = row.find('td', {'class': 'mat-cell cdk-cell cdk-column-dateString mat-column-dateString ng-star-inserted'}).text
            time_label = time.strptime(time_place, "%I:%M %p")
            num_secs = time_label.tm_hour*3600 + time_label.tm_min*60
            x = datetime(year_number, month_number, day_number) + timedelta(seconds=num_secs)

            temp = row.find('td', {'class': 'mat-cell cdk-cell cdk-column-temperature mat-column-temperature ng-star-inserted'}).text
            temp = int(temp.split('\xa0')[0])

            hum = row.find('td', {'class': 'mat-cell cdk-cell cdk-column-humidity mat-column-humidity ng-star-inserted'}).text
            hum = int(hum.split('\xa0')[0])

            precip = row.find('td', {'class': 'mat-cell cdk-cell cdk-column-precipRate mat-column-precipRate ng-star-inserted'}).text
            precip = float(precip.split('\xa0')[0])

            rows_data.append(
                [x, temp, hum, precip]       
            )

columns = ['Time', 'Temperature', 'Humidity', 'Precipitation']
df = pd.DataFrame(data=rows_data, columns=columns)
df.to_csv(F'Topsy Weather Scraper {year_number}.csv', index=False)   
df
        
#driver.quit()


In [246]:
driver.quit()